In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import datetime
from datetime import timedelta
import pymysql
from sqlalchemy import create_engine
# 导入模型
import xgboost as xgb

#模型调参的工具
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import GridSearchCV
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
#Error metrics评价指标
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pylab as plt

In [2]:
# 数据库信息
mysql_setting = {
    'host': '47.100.201.211',
    'port': 3306,
    'user': 'root',
    'passwd': 'iyGfLR64Ne4Ddhk7',
    # 数据库名称
    'db': 'data',
    'charset': 'utf8'
}
engine = create_engine("mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting), max_overflow=5)

In [3]:
def mov_df(df, num):
    df['mov_ori_avg'] = df['orign_price'].transform(lambda x: x.rolling(num).mean())
    df['mov_ori_max'] = df['orign_price'].transform(lambda x: x.rolling(num).max())
    df['mov_ori_min'] = df['orign_price'].transform(lambda x: x.rolling(num).min())
    df['mov_ori_var'] = df['orign_price'].transform(lambda x: x.rolling(num).var())
    return df
def mov_rate(df):
    df['滑窗产地价格波动'] = df['mov_ori_avg'].diff(periods=1)
    l1 = df['mov_ori_avg'].tolist()
    l2 = df['滑窗产地价格波动'].tolist()
    def rate(list1, list2):
        l = ['','','','','']
        i = 5
        while i <= len(list2)-1:
            r = list2[i]/list1[i-1]
            l.append(r)
            i = i + 1
        return l
    df['滑窗产地价格波动率'] = rate(l1, l2)
    df['滑窗产地价格波动率'] = pd.to_numeric(df['滑窗产地价格波动率'])
    return df
def train(df):
    x_train = df.iloc[5:-15,2:]
    y_train = df.iloc[5:-15,1]
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    x_predict = df.iloc[-15:,2:]
    x_predict = x_predict.reset_index(drop=True)
    return [x_train, y_train, x_predict]

In [4]:
def predict_offline_short(df):
    df.reset_index(level=0, inplace=True)
    df.rename(columns={'时间':'date','当日价格':'orign_price'},inplace=True)
    mov_df(df, 5)
    mov_rate(df)
    x_train = train(df)[0]
    y_train = train(df)[1]
    x_predict = train(df)[2]
    rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)
    boruta = BorutaPy(
       estimator = rf, 
       n_estimators = 'auto',
       max_iter = 100 # number of trials to perform
    )
    # 模型训练
    boruta.fit(np.array(x_train), np.array(y_train))
    # 输出结果
    green_area = x_train.columns[boruta.support_].to_list()
    blue_area  = x_train.columns[boruta.support_weak_].to_list()
    # 选择最优参数
    #重新选择特征
    f = []
    f.extend(green_area)
    f.extend(blue_area)
    x_train = x_train[f]
    x_predict = x_predict[f]
    # setup regressor
    xgb_model = xgb.XGBRegressor() 
    # performance a grid search
    tweaked_model = GridSearchCV(
        xgb_model,   
        {
            'max_depth':[1,2,5,10,20],
            'n_estimators':[20,30,50,70,100],
            'learning_rate':[0.1,0.2,0.3,0.4,0.5]
        },   
        cv = 3,   
        verbose = 1,
        n_jobs = -1,  
        scoring = 'neg_median_absolute_error')
    tweaked_model.fit(x_train, y_train)
    model1 = xgb.XGBRegressor(learning_rate = tweaked_model.best_params_['learning_rate'], 
                              max_depth = tweaked_model.best_params_['max_depth'], 
                              n_estimators = tweaked_model.best_params_['n_estimators'])
    model1.fit(x_train,y_train)
    Y_predict = model1.predict(x_predict).round(2)

    start_date = datetime.datetime.now().strftime('%Y-%m-%d')
    end_date = (datetime.datetime.now() + timedelta(days=16)).strftime('%Y-%m-%d')
    pre_date = pd.date_range(start = start_date, end = end_date)
    predict = pd.DataFrame(list(zip(pre_date,Y_predict)), columns = ['date','price'])
    return predict

In [5]:
# 中垾番茄
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.fanqie where 市场 like '%合肥%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
fanqie = dt.loc[dt['市场']=='合肥周谷堆农产品批发市场股份有限公司'].groupby('时间').mean('当日价格')

# 万年贡米，奉新大米，井岗红米，丰城大米，巢湖大米，仙桃香米
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.canmi"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
wanniangongmi = dt.loc[dt['市场']=='萍乡市安源春蕾农副产品发展有限公司'].groupby('时间').mean('当日价格')
fengxindami = dt.loc[dt['市场']=='新余市同盛实业集团有限责任公司'].groupby('时间').mean('当日价格')
jingganghongmi = dt.loc[dt['市场']=='南昌深圳农产品中心批发市场有限公司'].groupby('时间').mean('当日价格')
fengchengdami = dt.loc[dt['市场']=='江西崇仁江贸批发部'].groupby('时间').mean('当日价格')
chaohudami = dt.loc[dt['市场']=='合肥周谷堆农产品批发市场股份有限公司'].groupby('时间').mean('当日价格')
xiantaoxiangmi = dt.loc[dt['市场']=='武汉白沙洲农副产品大市场有限公司'].groupby('时间').mean('当日价格')

# 江西紫皮大蒜 
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.dasuan where 地区 like '江西%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
jiangxidasuan = dt.groupby('时间').mean('当日价格')

#沔城藕
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.lianou where 地区 like '湖北%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
mianchenou = dt.loc[dt['市场']=='襄阳竹叶山洪沟投资有限公司'].groupby('时间').mean('当日价格')

#永新酱姜， 上高白肉姜
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.shengjiang where 地区 like '江西%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
yongxinjiangjiang = dt.loc[dt['市场']=='乐平蔬菜批发大市场'].groupby('时间').mean('当日价格')
shanggaobairoujiang = dt.loc[dt['市场']=='萍乡市安源春蕾农副产品发展有限公司'].groupby('时间').mean('当日价格')

In [6]:
predict = predict_offline_short(fanqie)
predict['product'] = '中垾番茄'
order = ['product','date','price']
predict = predict[order]
df1 = predict

predict = predict_offline_short(chaohudami)
predict['product'] = '巢湖大米'
order = ['product','date','price']
predict = predict[order]
df2 = predict

predict = predict_offline_short(wanniangongmi)
predict['product'] = '万年贡米'
order = ['product','date','price']
predict = predict[order]
df3 = predict

predict = predict_offline_short(fengxindami)
predict['product'] = '奉新大米'
order = ['product','date','price']
predict = predict[order]
df4 = predict

predict = predict_offline_short(jingganghongmi)
predict['product'] = '井岗红米'
order = ['product','date','price']
predict = predict[order]
df5 = predict

predict = predict_offline_short(fengchengdami)
predict['product'] = '丰城大米'
order = ['product','date','price']
predict = predict[order]
df6 = predict

predict = predict_offline_short(jiangxidasuan)
predict['product'] = '江西紫皮大蒜'
order = ['product','date','price']
predict = predict[order]
df7 = predict

predict = predict_offline_short(mianchenou)
predict['product'] = '沔城藕'
order = ['product','date','price']
predict = predict[order]
df8 = predict

predict = predict_offline_short(xiantaoxiangmi)
predict['product'] = '仙桃香米'
order = ['product','date','price']
predict = predict[order]
df9 = predict

predict = predict_offline_short(yongxinjiangjiang)
predict['product'] = '永新酱姜'
order = ['product','date','price']
predict = predict[order]
df10 = predict

predict = predict_offline_short(shanggaobairoujiang)
predict['product'] = '上高白肉姜'
order = ['product','date','price']
predict = predict[order]
df11 = predict

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:   16.1s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    6.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    7.4s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    4.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    4.4s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    7.1s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 293 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    7.6s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [7]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]
df = reduce(lambda x, y: x.append(y), dfs)

In [8]:
sql = 'TRUNCATE TABLE predict_offline_short'
result = engine.execute(sql)

In [9]:
table_name = 'predict_offline_short'
df.to_sql(table_name, engine, if_exists='append', index=False)

<h1 align = center>商务部长期预测</h1>

In [13]:
def mov_df(df, num):
    df['mov_ori_avg'] = df['orign_price'].transform(lambda x: x.rolling(num).mean())
    df['mov_ori_max'] = df['orign_price'].transform(lambda x: x.rolling(num).max())
    df['mov_ori_min'] = df['orign_price'].transform(lambda x: x.rolling(num).min())
    df['mov_ori_var'] = df['orign_price'].transform(lambda x: x.rolling(num).var())
    return df
def mov_rate(df):
    df['滑窗产地价格波动'] = df['mov_ori_avg'].diff(periods=1)
    l1 = df['mov_ori_avg'].tolist()
    l2 = df['滑窗产地价格波动'].tolist()
    def rate(list1, list2):
        l = ['','','','','']
        i = 5
        while i <= len(list2)-1:
            r = list2[i]/list1[i-1]
            l.append(r)
            i = i + 1
        return l
    df['滑窗产地价格波动率'] = rate(l1, l2)
    df['滑窗产地价格波动率'] = pd.to_numeric(df['滑窗产地价格波动率'])
    return df
def train(df):
    x_train = df.iloc[5:-12,2:]
    y_train = df.iloc[5:-12,1]
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    x_predict = df.iloc[-12:,2:]
    x_predict = x_predict.reset_index(drop=True)
    return [x_train, y_train, x_predict]
def predict_offline_long(df):
    df.reset_index(level=0, inplace=True)
    df.rename(columns={'时间':'date','当日价格':'orign_price'},inplace=True)
    def deleteErrorDate(df):
        df.drop(df[df['date']=='2019-02-29'].index, inplace= True)
        df.drop(df[df['date']=='2019-02-30'].index, inplace= True)
        df.drop(df[df['date']=='2019-02-31'].index, inplace= True)    
        df.drop(df[df['date']=='2020-02-30'].index, inplace= True)  
        df.drop(df[df['date']=='2020-02-31'].index, inplace= True)  
        df.drop(df[df['date']=='2021-02-29'].index, inplace= True)  
        df.drop(df[df['date']=='2021-02-30'].index, inplace= True)  
        df.drop(df[df['date']=='2021-02-31'].index, inplace= True) 
        df.drop(df[df['date']=='2022-02-29'].index, inplace= True)  
        df.drop(df[df['date']=='2022-02-30'].index, inplace= True)  
        df.drop(df[df['date']=='2022-02-31'].index, inplace= True) 
        df.drop(df[df['date']=='2021-04-31'].index, inplace= True) 
        return df
    df = deleteErrorDate(df)
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime("%Y-%m")
    df = df.groupby('date').mean('orign_price').round(2)
    df.reset_index(level=0, inplace=True)
    mov_df(df, 5)
    mov_rate(df)
    x_train = train(df)[0]
    y_train = train(df)[1]
    x_predict = train(df)[2]
    rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)
    boruta = BorutaPy(
       estimator = rf, 
       n_estimators = 'auto',
       max_iter = 100 # number of trials to perform
    )
    # 模型训练
    boruta.fit(np.array(x_train), np.array(y_train))
    # 输出结果
    green_area = x_train.columns[boruta.support_].to_list()
    blue_area  = x_train.columns[boruta.support_weak_].to_list()
    # 选择最优参数
    #重新选择特征
    f = []
    f.extend(green_area)
    f.extend(blue_area)
    x_train = x_train[f]
    x_predict = x_predict[f]
    # setup regressor
    xgb_model = xgb.XGBRegressor() 
    # performance a grid search
    tweaked_model = GridSearchCV(
        xgb_model,   
        {
            'max_depth':[1,2,5,10,20],
            'n_estimators':[20,30,50,70,100],
            'learning_rate':[0.1,0.2,0.3,0.4,0.5]
        },   
        cv = 3,   
        verbose = 1,
        n_jobs = -1,  
        scoring = 'neg_median_absolute_error')
    tweaked_model.fit(x_train, y_train)
    model1 = xgb.XGBRegressor(learning_rate = tweaked_model.best_params_['learning_rate'], 
                              max_depth = tweaked_model.best_params_['max_depth'], 
                              n_estimators = tweaked_model.best_params_['n_estimators'])
    model1.fit(x_train,y_train)
    Y_predict = model1.predict(x_predict).round(2)
    pre_date = pd.period_range(datetime.datetime.now(), periods=12, freq='M')
    predict = pd.DataFrame(list(zip(pre_date,Y_predict)), columns = ['date','price'])
    return predict

In [14]:
# 中垾番茄
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.fanqie where 市场 like '%合肥%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
fanqie = dt.loc[dt['市场']=='合肥周谷堆农产品批发市场股份有限公司'].groupby('时间').mean('当日价格')

# 万年贡米，奉新大米，井岗红米，丰城大米，巢湖大米，仙桃香米
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.canmi"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
wanniangongmi = dt.loc[dt['市场']=='萍乡市安源春蕾农副产品发展有限公司'].groupby('时间').mean('当日价格')
fengxindami = dt.loc[dt['市场']=='新余市同盛实业集团有限责任公司'].groupby('时间').mean('当日价格')
jingganghongmi = dt.loc[dt['市场']=='南昌深圳农产品中心批发市场有限公司'].groupby('时间').mean('当日价格')
fengchengdami = dt.loc[dt['市场']=='江西崇仁江贸批发部'].groupby('时间').mean('当日价格')
chaohudami = dt.loc[dt['市场']=='合肥周谷堆农产品批发市场股份有限公司'].groupby('时间').mean('当日价格')
xiantaoxiangmi = dt.loc[dt['市场']=='武汉白沙洲农副产品大市场有限公司'].groupby('时间').mean('当日价格')

# 江西紫皮大蒜 
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.dasuan where 地区 like '江西%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
jiangxidasuan = dt.groupby('时间').mean('当日价格')

#沔城藕
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.lianou where 地区 like '湖北%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
mianchenou = dt.loc[dt['市场']=='襄阳竹叶山洪沟投资有限公司'].groupby('时间').mean('当日价格')

#永新酱姜， 上高白肉姜
sql_cmd = "SELECT 时间,市场,当日价格 FROM data.shengjiang where 地区 like '江西%'"
dt = pd.read_sql(sql=sql_cmd, con=engine,parse_dates=0,coerce_float=2)
yongxinjiangjiang = dt.loc[dt['市场']=='乐平蔬菜批发大市场'].groupby('时间').mean('当日价格')
shanggaobairoujiang = dt.loc[dt['市场']=='萍乡市安源春蕾农副产品发展有限公司'].groupby('时间').mean('当日价格')

In [15]:
predict = predict_offline_long(fanqie)
predict['product'] = '中垾番茄'
order = ['product','date','price']
predict = predict[order]
df1 = predict

predict = predict_offline_long(chaohudami)
predict['product'] = '巢湖大米'
order = ['product','date','price']
predict = predict[order]
df2 = predict

predict = predict_offline_long(wanniangongmi)
predict['product'] = '万年贡米'
order = ['product','date','price']
predict = predict[order]
df3 = predict

predict = predict_offline_long(fengxindami)
predict['product'] = '奉新大米'
order = ['product','date','price']
predict = predict[order]
df4 = predict

predict = predict_offline_long(jingganghongmi)
predict['product'] = '井岗红米'
order = ['product','date','price']
predict = predict[order]
df5 = predict

predict = predict_offline_long(fengchengdami)
predict['product'] = '丰城大米'
order = ['product','date','price']
predict = predict[order]
df6 = predict

predict = predict_offline_long(jiangxidasuan)
predict['product'] = '江西紫皮大蒜'
order = ['product','date','price']
predict = predict[order]
df7 = predict

# predict = predict_offline_long(xiantaoxiangmi)
# predict['product'] = '仙桃香米'
# order = ['product','date','price']
# predict = predict[order]
# df9 = predict

# predict = predict_offline_long(yongxinjiangjiang)
# predict['product'] = '永新酱姜'
# order = ['product','date','price']
# predict = predict[order]
# df10 = predict

# predict = predict_offline_long(mianchenou)
# predict['product'] = '沔城藕'
# order = ['product','date','price']
# predict = predict[order]
# df8 = predict

# predict = predict_offline_long(shanggaobairoujiang)
# predict['product'] = '上高白肉姜'
# order = ['product','date','price']
# predict = predict[order]
# df11 = predict

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    2.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    2.2s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    2.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    2.1s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    2.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    2.1s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.9s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    2.2s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [16]:
dfs = [df1, df2, df3, df4, df5, df6, df7]
df = reduce(lambda x, y: x.append(y), dfs)
df['date'] = df['date'].apply(lambda x: str(x))
df['date'] = pd.to_datetime(df['date'])

In [17]:
sql = 'TRUNCATE TABLE predict_offline_long'
result = engine.execute(sql)

In [18]:
table_name = 'predict_offline_long'
df.to_sql(table_name, engine, if_exists='append', index=False)